In [1]:
using LinearAlgebra, RowEchelon
using PyCall
itikz = pyimport("itikz")
nM    = pyimport("itikz.nicematrix")
jinja = pyimport("jinja2");

<div style="float:center;width:100%;text-align: center;"><strong style="height:100px;color:darkred;font-size:40px;">Fundamental Theorem, Complex Case</strong></div>

# 1. The Complex Inner Product Requires a Complex Conjugate

The complex inner product is given by $< u,v > = u \cdot \overline{v}$

Let $\mathscr{N}(A) = \left\{ x \mid A x = 0 \right\}$ be the usual definition of the null space.<br>
To verify orthogonality of a row space vector $r$ and a null space vector $x$,
we need $r \cdot \overline{x} = 0 \Leftrightarrow \overline{r} \cdot x = 0$.

It follows that $\mathscr{R}(\overline{A}) \perp \mathscr{N}(A)$.

We can apply this using various matrices:
* $\mathscr{R}({A}) \;\; \perp \mathscr{N}(\overline{A}) \qquad$ using $\overline{A}$.
* $\mathscr{R}({A^t}) \;\ \perp \mathscr{N}(A^H) \quad\;$ using $A^H$.
* $\mathscr{R}( \overline{A} ) \;\; \perp \mathscr{N}(A) \qquad$ using $A$.
* $\mathscr{R}({A^H}) \perp \mathscr{N}(A^t) \quad\;\;\ $ using $A^t$.


Similarly, we can convert statements about row spaces to statements about column spaces:
* $\mathscr{C}({A}) \;\; \perp \mathscr{N}(A^H) \qquad$ using $\overline{A}$.
* $\mathscr{C}({A^t}) \;\ \perp \mathscr{N}(\overline{A}) \qquad\;$ using $A^H$.
* $\mathscr{C}( \overline{A} ) \;\; \perp \mathscr{N}(A^t) \qquad\;$ using $A$.
* $\mathscr{C}({A^H}) \perp \mathscr{N}(A) \qquad\;\;$ using $A^t$.

### Simple Rank 1 Example

In [2]:
# Example Matrix
A = [1+1im 2+1im;-1+1im -1+2im]   # A
Ac = conj(A)                      # conjugate of A
At = conj(A')                     # transpose of A
Ah = A'                           # hermitian transpose of A

display(A)

2×2 Matrix{Complex{Int64}}:
  1+1im   2+1im
 -1+1im  -1+2im

#### Row and Column Spaces

In [3]:
# Row and column spaces
R_A  = A[ 1,:];    C_A  = A[ :,1]
R_Ac = Ac[1,:];    C_Ac = Ac[:,1]
R_At = At[1,:];    C_At = At[:,1]
R_Ah = Ah[1,:];    C_Ah = Ah[:,1]

2-element Vector{Complex{Int64}}:
 1 - 1im
 2 - 1im

#### Right and Left Null Spaces

In [4]:
# Right and Left Null Spaces
N_A  = [-3+1im;2]
N_Ac = [-3-1im;2]

N_At = [-1im;1]
N_Ah = conj(N_At)

2-element Vector{Complex{Int64}}:
 0 + 1im
 1 + 0im

#### Orthogonal Spaces

In [5]:
# orthogonalities: Note julia dot(u,v) is the complex inner product, not the dot product!

println("ℛ(A)  ⟂ 𝒩(Ac) : ", dot(R_A,  N_Ac) == 0)
println("ℛ(𝐴c) ⟂ 𝒩(A)  : ", dot(R_Ac, N_A ) == 0)
println("ℛ(At) ⟂ 𝒩(Ah) : ", dot(R_At, N_Ah) == 0)
println("ℛ(Ah) ⟂ 𝒩(At) : ", dot(R_Ah, N_At) == 0)
println()
println("𝒞(A)  ⟂ 𝒩(Ah) : ", dot(C_A,  N_Ah) == 0)
println("𝒞(Ac) ⟂ 𝒩(At) : ", dot(C_Ac, N_At) == 0)
println("𝒞(At) ⟂ 𝒩(Ac) : ", dot(C_At, N_Ac) == 0)
println("𝒞(Ah) ⟂ 𝒩(A)  : ", dot(C_Ah, N_A ) == 0)

ℛ(A)  ⟂ 𝒩(Ac) : true
ℛ(𝐴c) ⟂ 𝒩(A)  : true
ℛ(At) ⟂ 𝒩(Ah) : true
ℛ(Ah) ⟂ 𝒩(At) : true

𝒞(A)  ⟂ 𝒩(Ah) : true
𝒞(Ac) ⟂ 𝒩(At) : true
𝒞(At) ⟂ 𝒩(Ac) : true
𝒞(Ah) ⟂ 𝒩(A)  : true


# 2. Code

In [6]:
function tx(A) #transpose of a complex matrix
    conj(A')
end
function show(A)
    display( Complex{Int64}.(A))
end;

In [7]:
function find_pivot(A, row, col)
    for i in row:size(A,1)
        if A[i,col] != 0  return i end
    end
    -1
end
function non_zero_entry( A, row, col, gj )
    set = (row+1):size(A,1)
    if gj && row > 1
        set = [1:row-1; set]
    end
    for i in set
        if  A[i,col] != 0 return true end
    end
    false
end
function interchange(A, row_1, row_2)
    for j in 1:size(A,2)
        A[row_1,j],A[row_2,j] = A[row_2,j],A[row_1,j]
    end
end
function eliminate( A, pivot_row, row, alpha)
    for j in 1:size(A,2)
        A[row,j] += alpha * A[pivot_row,j]
    end
end

function e_ref(A; gj=false)
    matrices       = [[ :none, A ]]
    pivot_indices  = []
    if eltype(A) == Complex{Int64}
        A = Complex{Rational{Int64}}.(copy(A))
    elseif eltype(A) == Int64
        A = Rational{Int64}.(copy(A))
    else
        A = copy(A)  # caller took care of the type
    end

    M,N            = size(A)
    row = 1; col = 1
    while true
        if (row > M) || (col > N)
            if gj && M > 0                            # Scaling Matrix; only needed if there is a pivot != 1
                require_scaling = false

                E = Matrix{eltype(A)}(I, M, M)
                for i in 1:size(pivot_indices,1)
                    if isone( A[i,pivot_indices[i]] ) == false
                        require_scaling = true
                    end

                    E[i,i] = 1 // A[i,pivot_indices[i]] 
                end
                if require_scaling
                    push!(matrices, [E, E*A])
                end
            end
            return A, pivot_indices
        end

        p = find_pivot(A, row, col)
        if p < 0
            col += 1
        else
            push!(pivot_indices, col)
            if p != row
                interchange( A, p, row )
                E = Matrix{eltype(A)}( I, M, M)
                interchange( E, p, row )
                push!(matrices, [E, copy(A)])
            end

            if non_zero_entry( A, row, col, gj )
                E = Matrix{eltype(A)}(I, M, M)

                for r in (row+1):M
                    alpha = -A[r,col] / A[row,col]
                    eliminate(A, row, r, alpha )
                    eliminate(E, row, r, alpha )
                end

                if gj
                    for r in 1:(row-1)
                        alpha = -A[r,col] / A[row,col]
                        eliminate(A, row, r, alpha )
                        eliminate(E, row, r, alpha )
                    end
                end

                push!(matrices, [E, copy(A)])
            end
            col += 1; row += 1
        end
    end
    A, pivot_indices
end

function factor_out_denominator( A::Array{Rational{Int64},2} )
    d = reduce( lcm, denominator.(A) )
    d, Int64.(d*A)
end

function homogeneous_solutions( R, pivot_indices)
    # homogeneous solution from a reduced row echelon form R
    r = length(pivot_indices)                                                 # rank
    c = findall( j->j==1, [i in pivot_indices ? 0 : 1 for i in 1:size(R,2)] ) # free variable columns
    H = zeros(eltype(R), (size(R,2),length(c)))                               # matrix of homogeneous solutions
    for j in eachindex( c )                                                   # homogeneous solution vector x_j
        H[c[j],j] = 1                                                         # set the current free variable entry to 1
        H[pivot_indices,j] = -R[1:r, c[j]]                                    # set the pivot variable values
    end
    H
end
;

In [8]:
# Given two matrices, check that cols in U are orthogonal to cols in V, i.e., conj(u)^t v == 0
function check_orthogonal( U, V )
    orthogonal = true
    for i in 1:size(U)[2]
        for j in size(V)[2]
            orthogonal = orthogonal && dot(U[:,i], V[:,j]) == 0   # since dot(u,v) computes conj(u)^T v rather than u^T v
        end
    end

    orthogonal
end;

# 3. Example

In [9]:
U= [1    0  2 0 3;
    0    1 -1 2 1;
    0    0  0 0 0]+
1im*[ 0  0 -2 1 1;
      0  0  3 -1 2;
      0  0  0 0 0]
E = [ 1 0 0;
      2-3im 1 0; -2+1im 2-3im 1]
A=E*U
;

In [10]:
println("A =")
display(Complex{Int64}.(A))
R, pivot_indices   = e_ref(A, gj=true)
Rh,pivot_indices_h = e_ref(A',gj=true)
println()
println("R =")
display(Complex{Int64}.(R))

A =


3×5 Matrix{Complex{Int64}}:
  1+0im  0+0im   2-2im   0+1im    3+1im
  2-3im  1+0im  -3-7im   5+1im   10-5im
 -2+1im  2-3im   5+15im  0-10im   1+2im


R =


3×5 Matrix{Complex{Int64}}:
 1+0im  0+0im   2-2im  0+1im  3+1im
 0+0im  1+0im  -1+3im  2-1im  1+2im
 0+0im  0+0im   0+0im  0+0im  0+0im

In [11]:
Ac = conj(A)                      # conjugate of A
At = tx(A)                        # transpose of A
Ah = A'                           # hermitian transpose of A

# Define the spaces by writing bases into matrices as COLUMNS

R_A  = tx(R[ 1:length(pivot_indices),:]);          C_A  = A[ :,pivot_indices]   # basis of R_A as columns: tx(R_A)
R_Ac = conj(R_A);                                  C_Ac = conj(C_A)

R_At = tx(conj(Rh[ 1:length(pivot_indices),:]));   C_At = At[:, pivot_indices_h] # keep bases as columns: tx(tx(R_A)), tx(tx(Ca))
R_Ah = conj(R_At);                                 C_Ah = conj(C_At)               # keep bases as columns: tx(R_A'), tx(Ca')

N_A  = homogeneous_solutions( R, pivot_indices )
N_Ac = conj(N_A)
N_Ah = homogeneous_solutions( Rh, pivot_indices_h )
N_At = conj(N_Ah)                                                                  # keep basis as columns: tx(tx(Na))
Complex{Int64}.(Ah*N_Ah)

5×1 Matrix{Complex{Int64}}:
 0 + 0im
 0 + 0im
 0 + 0im
 0 + 0im
 0 + 0im

In [12]:
println("ℛ(A)  ⟂ 𝒩(Ac) : ", check_orthogonal(R_A,  N_Ac) )
println("ℛ(𝐴c) ⟂ 𝒩(A)  : ", check_orthogonal(R_Ac, N_A ) )
println("ℛ(At) ⟂ 𝒩(Ah) : ", check_orthogonal(R_At, N_Ah) )
println("ℛ(Ah) ⟂ 𝒩(At) : ", check_orthogonal(R_Ah, N_At) )
println()
println("𝒞(A)  ⟂ 𝒩(Ah) : ", check_orthogonal(C_A,   N_Ah) )
println("𝒞(Ac) ⟂ 𝒩(At) : ", check_orthogonal(C_Ac,  N_At) )
println("𝒞(At) ⟂ 𝒩(Ac) : ", check_orthogonal(C_At,  N_Ac) )
println("𝒞(Ah) ⟂ 𝒩(A)  : ", check_orthogonal(C_Ah,  N_A ) )

ℛ(A)  ⟂ 𝒩(Ac) : true
ℛ(𝐴c) ⟂ 𝒩(A)  : true
ℛ(At) ⟂ 𝒩(Ah) : true
ℛ(Ah) ⟂ 𝒩(At) : true

𝒞(A)  ⟂ 𝒩(Ah) : true
𝒞(Ac) ⟂ 𝒩(At) : true
𝒞(At) ⟂ 𝒩(Ac) : true
𝒞(Ah) ⟂ 𝒩(A)  : true


In [13]:
######### RENEW LIBRARY CARD
######### Heating Duct Filter